In [1]:
import os
import json
import numpy as np
import shutil
import random
import cv2 as cv

resolution = 300 #it means the images will be downscaled to 300x300

In [2]:
#some supporting functions
def normalize_images_size(source_path, target_path, file_type, target_width, target_height, default_background_color=0,
                          verbose=False):
    file_names = [f for f in os.listdir(source_path)
                  if os.path.isfile(os.path.join(source_path, f)) and f.endswith(file_type)]

    for file_name in file_names:
        if verbose:
            print('Converting ' + file_name)
        source_image = cv.imread(os.path.join(source_path,file_name), cv.IMREAD_COLOR)
        result_image = normalize_image_size(source_image, target_width, target_height, default_background_color)
        cv.imwrite(os.path.join(target_path, file_name), result_image)


def normalize_image_size(image, target_width, target_height, default_background_color=0):
    height, width, _ = image.shape
    height_ratio = height / target_height
    width_ratio = width / target_width
    if width_ratio > height_ratio:
        ret = __resize_by_width(image, target_width)
    else:
        ret = __resize_by_height(image, target_height)

    height, width, _ = ret.shape
    if height < target_height or width < target_width:
        ret = __extend_to_exact_size(ret, target_width, target_height, default_background_color)

    return ret


def __resize_by_width(img, target_width):
    current_height, current_width, _ = img.shape
    target_height = int(target_width / current_width * current_height)
    ret = cv.resize(img, (target_width, target_height), interpolation=cv.INTER_LINEAR)
    return ret


def __resize_by_height(img, target_height):
    current_height, current_width, _ = img.shape
    target_width = int(target_height / current_height * current_width)
    ret = cv.resize(img, (target_width, target_height), interpolation=cv.INTER_LINEAR)
    return ret


def __extend_to_exact_size(img, target_width, target_height, default_background_color):
    current_height, current_width, _ = img.shape
    left_border = int((target_width - current_width) / 2)
    right_border = target_width - current_width - left_border
    top_border = int((target_height - current_height) / 2)
    bottom_border = target_height - current_height - top_border
    ret = cv.copyMakeBorder(img, top_border, bottom_border, left_border, right_border,
                            cv.BORDER_CONSTANT, None, default_background_color)
    return ret

def is_intersect(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    if interArea > 0: return True
    return False

In [3]:
out_dir       = r'D:\SIGNATE\Signate_3rd_AI_edge_competition\classifier_images\train' #the directory has to be created!
out_dir_val   = r'D:\SIGNATE\Signate_3rd_AI_edge_competition\classifier_images\val' #the directory has to be created!
path_labels   = r'D:\SIGNATE\Signate_3rd_AI_edge_competition/data_for_yolo_training.txt'
generate_random = 12 #negative images

with open(path_labels) as f:  lines    = f.readlines()
for i in range (0, len(lines)): lines[i] = lines[i].split()
f.close()

counts = np.asarray((0, 0, 0))


bad_boxes = 0
good_boxes = 0
extreme_bad_boxes = 0
for i in range (0, len(lines)):
    #if i % 500==1: print(i, len(lines))
        
    img   = cv.imread(lines[i][0])
    boxes = np.array([np.array(list(map(int, box.split(',')))) for box in lines[i][1:]])
    
    #add standard crops
    for b in range (0, len(boxes)):
        cls = boxes[b][4]
        if cls>1: cls = 2
        
        px1 = max(boxes[b][0], 0)
        py1 = max(boxes[b][1], 0)
        px2 = min(boxes[b][2], img.shape[1])
        py2 = min(boxes[b][3], img.shape[0])
        
        if (px2-px1)*(py2-py1)<800: #we do not want small boxes
            continue
        
        if px2-px1<10 or py2-py1<10:
            bad_boxes = bad_boxes+1
            if px2-px1<=0 or py2-py1<=0:
                extreme_bad_boxes = extreme_bad_boxes + 1
            continue
        good_boxes = good_boxes+1

        sub = normalize_image_size(img[py1:py2, px1:px2, :], resolution, resolution, 0)
        
        
        #２０個に１個Validationデータへ
        if i%20==1:
            print("\r i={}, b={} :".format(i, b) + out_dir_val+'/'+str(cls)+'/'+str(cls)+'_'+str(counts[cls])+'.jpg           ', end="")
            cv.imwrite(out_dir_val+'/'+str(cls)+'/'+str(cls)+'_'+str(counts[cls])+'.jpg', sub)
        else:
            print("\r i={}, b={} :".format(i, b) + out_dir+'/'+str(cls)+'/'+str(cls)+'_'+str(counts[cls])+'.jpg           ', end="")
            cv.imwrite(out_dir+'/'+str(cls)+'/'+str(cls)+'_'+str(counts[cls])+'.jpg', sub)
        counts[cls]+= 1
        
    #print("\n")
    # 背景画像作成
    #add background crops
    act_gen = 0
    while act_gen<generate_random:
        start_x = random.randint(0,img.shape[1])
        start_y  = random.randint(0,img.shape[0])
        w = random.randint(10,img.shape[1]//2)
        h = random.randint(10,img.shape[0]//2)
        stop_x = w+start_x
        stop_y = h+start_y
        if stop_x > img.shape[1] or stop_y > img.shape[0]:
            continue
        
        if w/h > 2 or w/h<0.25: #we do not want so wide objects
            continue
            
        if w<10 or h<10: #some errors in labelings?
            continue
            
        if w*h < 500: #we do not want small images
            continue
            
        
        inter = False
        box = np.asarray((start_x, start_y, stop_x, stop_y))
        for b in range (0, len(boxes)):
            if is_intersect(box, np.asarray((boxes[b][0],boxes[b][1],boxes[b][2],boxes[b][3]))):
                inter = True
                break
        if inter:
            continue
        
        cls = 2
        sub = normalize_image_size(img[start_y:stop_y, start_x:stop_x, :], resolution, resolution, 0)
        if i%20==1:
            print("\r i={}, act_gen={} :".format(i, act_gen) + out_dir_val+'/'+str(cls)+'/'+str(cls)+'_'+str(counts[cls])+'.jpg           ', end="")
            cv.imwrite(out_dir_val+'/'+str(cls)+'/'+str(cls)+'_'+str(counts[cls])+'.jpg', sub)
        else:
            print("\r i={}, act_gen={} :".format(i, act_gen) + out_dir+'/'+str(cls)+'/'+str(cls)+'_'+str(counts[cls])+'.jpg           ', end="")
            cv.imwrite(out_dir+'/'+str(cls)+'/'+str(cls)+'_'+str(counts[cls])+'.jpg', sub)
        counts[cls]+= 1
            
        act_gen+=1
        

print('extreme bad boxes: ', extreme_bad_boxes)        
print('bad boxes: ', bad_boxes)    
print('good boxes: ', good_boxes)                                                                        

 i=34441, act_gen=11 :D:\SIGNATE\Signate_3rd_AI_edge_competition\classifier_images\val/2/2_413303.jpg           extreme bad boxes:  0
bad boxes:  7
good boxes:  389446
